In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Computes the value of the goal-attraction cost.
# Returns the cost
def goal_attraction(e_phi: np.ndarray, # Current end effector coords
                    g: np.ndarray # Goal end effector coords
                    ) -> float:
    c = np.linalg.norm(e_phi - g)
    return c

In [3]:
# Computes the value of the field potential for obstacle avoidance
# Returns the cost
def F_R(d: float # Distance between end effector and obstacle
        ) -> float:
    R = 50
    cost = 2
    if (d <= R) and (d > 0):
        cost = np.log(R/d)
    elif (d > R):
        cost = 0
    return cost

In [4]:
# Computes the value of the field potential for joint-limit cost
# Returns the cost
def L(phi: float # Joint angle
      ) -> float:
    phi_min = 0
    phi_max = 180
    delta = 10
    cost = 2
    
    if (phi > phi_min) and (phi <= (phi_min+delta)):
        cost = np.log(phi / (phi-phi_min))
    elif (phi > phi_min + delta) and (phi < phi_max - delta):
        cost = 0
    elif (phi >= phi_max - delta) and (phi < phi_max):
        cost = np.log(phi / (phi_max - phi))
    
    return cost

In [5]:
def end_effector(phi):
    theta_array = phi
    
    arm_position = [5, 5, 0]
    length_array = [2, 5, 5, 5]
    
    T_01 = np.array([[np.cos(theta_arr[0]), -np.sin(theta_arr[0]), 0.0, arm_position[0]],
                     [np.sin(theta_arr[0]), np.cos(theta_arr[0]) , 0.0, arm_position[1]],
                     [0.0                 , 0.0                  , 1.0, arm_position[2]],
                     [0.0                 , 0.0                  , 0.0,             1.0]]
                   )
    # Frame 1 written w.r.t. Frame 2
    T_12 = np.array([[np.cos(theta_arr[1]) , 0.0, np.sin(theta_arr[1]), 0.0],
                     [0.0                  , 1.0, 0.0                 , 0.0],
                     [-np.sin(theta_arr[1]), 0.0, np.cos(theta_arr[1]), length_arr[0]],
                     [0.0                  , 0.0, 0.0                 , 1.0]]
                   )
    # Frame 3 written w.r.t. Frame 2
    T_23 = np.array([[np.cos(theta_arr[2]) , 0.0, np.sin(theta_arr[2]), 0.0],
                     [0.0                  , 1.0, 0.0                 , 0.0],
                     [-np.sin(theta_arr[2]), 0.0, np.cos(theta_arr[2]), length_arr[1]],
                     [0.0                  , 0.0, 0.0                 , 1.0]]
                   )
    # Frame 4 written w.r.t. Frame 3
    T_34 = np.array([[np.cos(theta_arr[3]) , 0.0, np.sin(theta_arr[3]), 0.0],
                     [0.0                  , 1.0, 0.0                 , 0.0],
                     [-np.sin(theta_arr[3]), 0.0, np.cos(theta_arr[3]), length_arr[2]],
                     [0.0                  , 0.0, 0.0                 , 1.0]]
                   ) 
    
    # Local-to-global transformation matrices 
    T_04 = T_01 @ T_12 @ T_23 @ T_34                    # Frame 4 written w.r.t. Frame 0
    e = T_04[0:3, 3]
    
    return e 

In [6]:
# Minimize the cost
def cost_minimize():

    # Delta change in each angle
    deltas = [0.1, 0.1, 0.1, 0.1]

    # Delta change in each angle in matrix form, for easier addition
    deltas_matrix = np.array([
                      [0.1, 0, 0, 0],
                      [0, 0.1, 0, 0],
                      [0, 0, 0.1, 0],
                      [0, 0, 0, 0.1]])

    gradient = np.empty((1, 1))

    # For each angle in arm
    for i in range(4) :

        # Calculate components of gradient
        deltaC = ( C(Phi + deltas_matrix[i], goal) - C(Phi, goal) ) / deltas[i]

        # Add to gradient
        gradient.append(deltaC)

In [ ]:
# Cost function
def C(phi: float, # Angle
      goal_location: np.ndarray # Location of the goal
      ): 
    # Goal (target location) 
    g = np.array([[30],[50],[10]])

    # Current joint-angle configuration
    phi = np.array([[0],[11],[22],[33]])

    # End-effector location 
    e_phi = end_effector(phi)

    # Obstacles 
    num_obstacles = 3

    # Location of obstacles 
    obstacle = np.array([[10,10,10], 
                         [15,10,10], 
                         [20,20,10]])

    cost = 0

    # Goal attraction?
    cost = np.linalg.norm(e_phi - g) 
    print(cost)

    # Obstacle avoidance penalty
    cost += sum(F_R(np.linalg.norm(e_phi - obstacle[:,j])) for j in range(num_obstacles))
    print(cost)

    # Joint-range limit
    cost += sum(L(p) for p in phi)  
    print(cost)
    return cost

In [7]:
# Computes the value of the function at x
# Returns 3x4 Jacobian
def J(phi: float # Angle
      ):
    
    delta = 0.1    
    deltas = np.array([
                      [0.1, 0, 0, 0],
                      [0, 0.1, 0, 0],
                      [0, 0, 0.1, 0],
                      [0, 0, 0, 0.1]])
    
    J = np.empty((3, 1))
    
    for i in range(4): 
        e_delta = ( end_effector(phi + deltas[i]) - end_effector(phi))
        DxDphi = e_delta[0] / delta
        DyDphi = e_delta[1] / delta
        DzDphi = e_delta[2] / delta
        J = np.concatenate( (J, np.array([[DxDphi], [DyDphi], [DzDphi]])), axis=1)
    return J[:,1:5]


In [14]:
# Solves the inverse kinematic problem by using gradient descent
def gradient_descent_IK(pos, target_pos): 
    delta_f = target_pos - pos           # Difference between predicted and target
    dist = np.linalg.norm(delta_f)          # Distance from target
    phi = 
    jacobian = J(phi)              # Compute the Jacobian for the current pose
    jacobian_inverse = np.linalg.inv(jacobian)                 # Invert the Jacobian matrix
    delta_pos_mapped = jacobian_inverse @ delta_f       # Compute the change in pose
    f_predicted = f(pos + delta_pos_mapped) # Apply change to joint angles
    
    print(uv_g)
    return uv_g

In [15]:
current = np.array([[0],[0],[0]])
goal = np.array([[10],[10],[10]])
gradient_descent_IK(current, goal)

NameError: name 'phi' is not defined